<pre>
Copyright 2022 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to enable
# nested event loops. We don't need that when using `isabelle-client` from
# Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
# first, we start Isabelle server
from isabelle_client import start_isabelle_server

server_info, isabelle_process = start_isabelle_server()

In [3]:
# server info is the same as printed by `isabelle server` command
print(server_info)

server "isabelle" = 127.0.0.1:38145 (password "2b731e67-df0a-4493-a9c4-09d82ad14fc6")



In [4]:
# a process object can be used for additional control
type(isabelle_process)

asyncio.subprocess.Process

In [5]:
isabelle_process

<Process 1235>

In [6]:
isabelle_process.pid

1235

In [7]:
# now we can create a client instance
from isabelle_client import get_isabelle_client

isabelle = get_isabelle_client(server_info)
isabelle

In [8]:
# the server's commands are implemented as client object methods
responses = isabelle.help()
responses

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]', response_length=118)]

In [9]:
# usually, the last server's response contains something interesting
responses[-1]

IsabelleResponse(response_type='OK', response_body='["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]', response_length=118)

In [10]:
responses[-1].response_body

'["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]'

In [11]:
# the response body is usually JSON formatted
import json

data = json.loads(responses[-1].response_body)
data

['cancel',
 'echo',
 'help',
 'purge_theories',
 'session_build',
 'session_start',
 'session_stop',
 'shutdown',
 'use_theories']

In [12]:
%%time
# let's do something more meaningful

session_id = isabelle.session_start()
session_id

CPU times: user 10.6 ms, sys: 0 ns, total: 10.6 ms
Wall time: 17.7 s


'3a351829-9b53-4e33-8359-1ae92907af8c'

In [13]:
# let's generate a dummy problem file in Isabelle format
with open("Problem.thy", "w") as problem_file:
    problem_text = """
theory Problem imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
sledgehammer
oops
end
    """
    problem_file.write(problem_text)

In [14]:
%%time
# now let's ask the server to solve the problem

responses = isabelle.use_theories(
    theories=["Problem"],
    master_dir=".",
    session_id=session_id
)
responses

CPU times: user 9.65 ms, sys: 211 µs, total: 9.86 ms
Wall time: 15.6 s


[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"2142ac80-245f-4cfe-86c8-b4cbc713312f"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":18,"task":"2142ac80-245f-4cfe-86c8-b4cbc713312f","message":"theory Draft.Problem 18%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"2142ac80-245f-4cfe-86c8-b4cbc713312f","message":"theory Draft.Problem 99%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":100,"task":"2142ac80-245f-4cfe-86c8-b4cbc713312f","message":"theory Draft.Problem 100%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=163),
 IsabelleResponse(respons

In [15]:
# notice that important messages live inside "nodes" object
data = json.loads(responses[-1].response_body)
data

{'ok': True,
 'errors': [],
 'nodes': [{'messages': [{'kind': 'writeln',
     'message': 'Sledgehammering...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'Proof found...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': '"cvc4": Try this: by simp (1 ms)',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': '"verit": Try this: by presburger (1 ms)',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': '"z3": Try this: by simp (1 ms)',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': '"vampire": Try this: by simp (3 ms)',
     'pos': {'line': 5,


In [16]:
# the messages from `sledgehammer` have an easily parsable format
messages = [message["message"] for message in data["nodes"][0]["messages"]]
messages

['Sledgehammering...',
 'Proof found...',
 '"cvc4": Try this: by simp (1 ms)',
 '"verit": Try this: by presburger (1 ms)',
 '"z3": Try this: by simp (1 ms)',
 '"vampire": Try this: by simp (3 ms)',
 '"e": Try this: by force (5 ms)',
 '"spass": Try this: by blast (2 ms)']

In [17]:
# we can parse `sledgehammer` suggestions using regular expressions
import re

proposed_solutions = [
    re.findall('".*": Try this: (.*) \(\d+ ms\)', message)
    for message in messages
]
proposed_solutions

[[],
 [],
 ['by simp'],
 ['by presburger'],
 ['by simp'],
 ['by simp'],
 ['by force'],
 ['by blast']]

In [18]:
# in principle, we can value some solutions more
import random

solution = random.choice([
    proposed_solution[0]
    for proposed_solution in proposed_solutions
    if proposed_solution
])
solution

'by simp'

In [19]:
# now let's write down our solution to a file
with open("Solution.thy", "w") as solution_file:
    solution_text = (
        problem_text
        .replace("sledgehammer\noops\n", f"{solution}\n")
        .replace("theory Problem ", "theory Solution ")
    )
    print(solution_text)
    solution_file.write(solution_text)


theory Solution imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by simp
end
    


In [20]:
%%time
# and finally let's ask the server to check the solution

responses = isabelle.use_theories(
    theories=["Solution"],
    master_dir=".",
    session_id=session_id
)
responses

CPU times: user 10.4 ms, sys: 957 µs, total: 11.3 ms
Wall time: 14.9 s


[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"cc332841-911f-4293-acc5-6e9ee76ff335"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":22,"task":"cc332841-911f-4293-acc5-6e9ee76ff335","message":"theory Draft.Solution 22%","kind":"writeln","session":"","theory":"Draft.Solution"}', response_length=163),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"cc332841-911f-4293-acc5-6e9ee76ff335","message":"theory Draft.Solution 99%","kind":"writeln","session":"","theory":"Draft.Solution"}', response_length=163),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":100,"task":"cc332841-911f-4293-acc5-6e9ee76ff335","message":"theory Draft.Solution 100%","kind":"writeln","session":"","theory":"Draft.Solution"}', response_length=165),
 IsabelleResponse(r

In [21]:
# the response body doesn't contain any errors
json.loads(responses[-1].response_body)

{'ok': True,
 'errors': [],
 'nodes': [{'messages': [],
   'exports': [],
   'status': {'percentage': 100,
    'unprocessed': 0,
    'running': 0,
    'finished': 9,
    'failed': 0,
    'total': 9,
    'consolidated': True,
    'canceled': False,
    'ok': True,
    'warned': 0},
   'theory_name': 'Draft.Solution',
   'node_name': 'Solution.thy'}],
 'task': 'cc332841-911f-4293-acc5-6e9ee76ff335'}

In [22]:
# we can gracefully finish the session
isabelle.session_stop(session_id)

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"29c104d2-0035-4c56-9076-a532d00b5479"}', response_length=None),
 IsabelleResponse(response_type='FINISHED', response_body='{"ok":true,"return_code":0,"task":"29c104d2-0035-4c56-9076-a532d00b5479"}', response_length=None)]

In [23]:
# and shut the server down
isabelle.shutdown()

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='', response_length=None)]